In [1]:
%load_ext autoreload
%autoreload 2
    
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import pickle 
from matplotlib.path import Path
from PIL import Image
import cv2

import glob
import os
import h5py

from utils import get_matrix_points, get_coords


In [13]:
%reload_ext autoreload

In [2]:
ROWS, COLS = 18, 18

def process_webcam_images(path):
    img = Image.open(path).convert("L")

def get_mirror(m_points, idx, image):
    indices = [(i, j) for i in range(ROWS - 1) for j in range(COLS - 1)]
    N = len(indices)
    state = {'idx': idx}

    x, y = get_coords(state, indices, m_points)

    mirror = extract_polygon_region_cv2_from_image(image, x, y)
    return mirror

def extract_polygon_region_cv2_from_image(img, x_coords, y_coords):
    """
    Extracts a polygonal region from an image and returns it as a new (cropped) image.
    Args:
        img: Input image.
        x_coords, y_coords: Lists of x and y coordinates of polygon vertices.

    Returns:
        Cropped polygon region as a new image.
    """

    # Wczytaj obraz w skali szarości
    # img = np.array(Image.open(img_path).convert('L'))  # -> shape: (H, W), dtype: uint8
    
    # Przekształć punkty wielokąta
    pts = np.array(list(zip(x_coords, y_coords)), dtype=np.int32)
    pts = pts.reshape((-1, 1, 2))
    
    # Maska binarna o tym samym rozmiarze
    mask = np.zeros(img.shape, dtype=np.uint8)
    cv2.fillPoly(mask, [pts], color=255)
    
    # Nałożenie maski
    masked_img = cv2.bitwise_and(img, img, mask=mask)
    
    # Przycięcie do prostokąta ograniczającego
    x, y, w, h = cv2.boundingRect(pts)
    cropped = masked_img[y:y+h, x:x+w]

    return cropped

In [4]:
with open("good_images_name.txt", "r") as f:
    good_images_list = f.readlines()

In [15]:
DIR_PATH = "/media/pgliwny/ADATA HD3301/Computer_Vision_system/data/MAGIC/webcam"
IMAGE_LIST = []
for f_name in good_images_list[::5]:
    f_name.strip()
    image_path = os.path.join(DIR_PATH, f_name.strip())
    IMAGE_LIST.append(image_path)
    

In [16]:
len(IMAGE_LIST)

254

In [3]:
def mirrors_data_generator():
    # Example: load one image and its mirrors at a time
    for img_path in IMAGE_LIST:
        img_name = img_path.split("/")[-1]
        img = Image.open(img_path).convert("L")
        
        mirrors_list = [get_mirror(M_POINTS, i, np.array(img)) for i in range(289)]
    
        yield np.array(img), img_name, mirrors_list

In [17]:
M_POINTS = get_matrix_points()

# Open HDF5 file once (for writing)
with h5py.File("webcam_images_good.h5", "w") as f:
    images_group = f.create_group("images")

    for idx, (img_array, img_name, mirrors_list) in enumerate(mirrors_data_generator()):
        # Create a group for this entry
        entry_group = images_group.create_group(f"entry_{idx:04d}")
        
        # Save image
        entry_group.create_dataset("image", data=img_array)

        # Save name
        entry_group.create_dataset("image_name", data=img_name.encode("utf-8"))

        # Save mirrors
        mirrors_group = entry_group.create_group("mirrors")
        for m_idx, mirror in enumerate(mirrors_list):
            mirrors_group.create_dataset(f"mirror_{m_idx}", data=mirror)

In [ ]:
def mirrors_data_generator():
    # Example: load one image and its mirrors at a time
    for img_path in IMAGE_LIST:
        img_name = img_path.split("/")[-1]
        img = Image.open(img_path).convert("L")
        
        mirrors_list = [get_mirror(M_POINTS, i, np.array(img)) for i in range(289)]
    
        yield np.array(img), img_name, mirrors_list

In [13]:
WEBCAM_IMG_PATH = "/media/pgliwny/ADATA HD330/Computer_Vision_system/data/MAGIC/webcam/"
IMAGE_LIST = glob.glob(os.path.join(WEBCAM_IMG_PATH,"image_2024*1200*"))
print(IMAGE_LIST[:3])
print(len(IMAGE_LIST))

['/media/pgliwny/ADATA HD330/Computer_Vision_system/data/MAGIC/webcam/image_2024-06-21_1200.jpg', '/media/pgliwny/ADATA HD330/Computer_Vision_system/data/MAGIC/webcam/image_2024-01-02_1200.jpg', '/media/pgliwny/ADATA HD330/Computer_Vision_system/data/MAGIC/webcam/image_2024-01-03_1200.jpg']
203


In [14]:
M_POINTS = get_matrix_points()

# Open HDF5 file once (for writing)
with h5py.File("webcam_images.h5", "w") as f:
    images_group = f.create_group("images")

    for idx, (img_array, img_name, mirrors_list) in enumerate(mirrors_data_generator()):
        # Create a group for this entry
        entry_group = images_group.create_group(f"entry_{idx}")
        
        # Save image
        entry_group.create_dataset("image", data=img_array)

        # Save name
        entry_group.create_dataset("image_name", data=img_name.encode("utf-8"))

        # Save mirrors
        mirrors_group = entry_group.create_group("mirrors")
        for m_idx, mirror in enumerate(mirrors_list):
            mirrors_group.create_dataset(f"mirror_{m_idx}", data=mirror)

# Example with list

In [8]:
WEBCAM_IMG_PATH = "/media/pgliwny/ADATA HD330/Computer_Vision_system/data/MAGIC/webcam/"
image_list = glob.glob(os.path.join(WEBCAM_IMG_PATH,"image_2024*1200*"))
print(image_list[:3])
print(len(image_list))

['/media/pgliwny/ADATA HD330/Computer_Vision_system/data/MAGIC/webcam/image_2024-06-21_1200.jpg', '/media/pgliwny/ADATA HD330/Computer_Vision_system/data/MAGIC/webcam/image_2024-01-02_1200.jpg', '/media/pgliwny/ADATA HD330/Computer_Vision_system/data/MAGIC/webcam/image_2024-01-03_1200.jpg']
203


In [27]:
m_points = get_matrix_points()

In [ ]:
image_data_list = []
for img_path in image_list[:5]:
    img = Image.open(img_path).convert("L")
    print(img_path)
    mirrors_list = [get_mirror(m_points, i, np.array(img)) for i in range(289)]

    
    d = {
        "name":image_list[0].split("/")[-1],
        "image": np.array(img),
        "mirrors": mirrors_list
    }
    image_data_list.append(d)



/media/pgliwny/ADATA HD330/Computer_Vision_system/data/MAGIC/webcam/image_2024-01-09_1500.jpg
/media/pgliwny/ADATA HD330/Computer_Vision_system/data/MAGIC/webcam/image_2024-02-01_1800.jpg
/media/pgliwny/ADATA HD330/Computer_Vision_system/data/MAGIC/webcam/image_2024-02-25_1400.jpg
/media/pgliwny/ADATA HD330/Computer_Vision_system/data/MAGIC/webcam/image_2024-03-19_1700.jpg
/media/pgliwny/ADATA HD330/Computer_Vision_system/data/MAGIC/webcam/image_2024-04-12_1100.jpg
/media/pgliwny/ADATA HD330/Computer_Vision_system/data/MAGIC/webcam/image_2024-05-05_1400.jpg
/media/pgliwny/ADATA HD330/Computer_Vision_system/data/MAGIC/webcam/image_2024-05-28_1700.jpg
/media/pgliwny/ADATA HD330/Computer_Vision_system/data/MAGIC/webcam/image_2024-06-21_1200.jpg
/media/pgliwny/ADATA HD330/Computer_Vision_system/data/MAGIC/webcam/image_2024-01-01_1000.jpg
/media/pgliwny/ADATA HD330/Computer_Vision_system/data/MAGIC/webcam/image_2024-01-01_1100.jpg
/media/pgliwny/ADATA HD330/Computer_Vision_system/data/MAGIC

In [38]:
with h5py.File("webcam_images.h5", "w") as f:
    images_group = f.create_group("images")

    for idx, entry in enumerate(image_data_list):
        entry_group = images_group.create_group(f"entry_{idx}")
        entry_group.create_dataset("image", data=entry["image"])
        entry_group.create_dataset("image_name", data=entry["name"].encode("utf-8"))
        
        mirrors_group = entry_group.create_group("mirrors")
        for m_idx, mirror in enumerate(entry["mirrors"]):
            mirrors_group.create_dataset(f"mirror_{m_idx}", data=mirror)

# Example one image

In [19]:
WEBCAM_IMG_PATH = "/media/pgliwny/ADATA HD330/Computer_Vision_system/data/MAGIC/webcam/"
image_list = glob.glob(os.path.join(WEBCAM_IMG_PATH,"image_2024*"))
print(image_list[:3])

['/media/pgliwny/ADATA HD330/Computer_Vision_system/data/MAGIC/webcam/image_2024-01-09_1500.jpg', '/media/pgliwny/ADATA HD330/Computer_Vision_system/data/MAGIC/webcam/image_2024-02-01_1800.jpg', '/media/pgliwny/ADATA HD330/Computer_Vision_system/data/MAGIC/webcam/image_2024-02-25_1400.jpg']


In [ ]:
img = Image.open(image_list[0]).convert("L")

plt.imshow(np.array(img))

In [8]:
! ls

__pycache__  save_mirror_to_file.ipynb	utils.py


In [14]:
m_points = get_matrix_points()

In [15]:
m_points.shape

(18, 18)

In [16]:
m1 = get_mirror(m_points, 280, np.array(img))

In [20]:
mirrors_list = [get_mirror(m_points, i, np.array(img)) for i in range(289)]

In [21]:
m1.shape

(18, 16)

In [22]:
np.array(img).shape

(480, 830)

In [23]:
len(mirrors_list)

289

In [24]:
mirrors_list[0].shape

(10, 12)

In [25]:
mirrors_list[1].shape

(13, 17)

In [27]:
image_list[0].split("/")[-1]

'image_2024-01-09_1500.jpg'

In [22]:
image_data = [
    {
        "name":image_list[0].split("/")[-1],
        "image": np.array(img),
        "mirrors": mirrors_list
    }
]

In [23]:
with h5py.File("webcam_images.h5", "w") as f:
    images_group = f.create_group("images")

    for idx, entry in enumerate(image_data):
        entry_group = images_group.create_group(f"entry_{idx}")
        entry_group.create_dataset("image", data=entry["image"])
        entry_group.create_dataset("image_name", data=entry["name"].encode("utf-8"))
        
        mirrors_group = entry_group.create_group("mirrors")
        for m_idx, mirror in enumerate(entry["mirrors"]):
            mirrors_group.create_dataset(f"mirror_{m_idx}", data=mirror)
                                       

In [32]:
with h5py.File("webcam_images.h5", "r") as f:
    for entry_name in f["images"]:
        entry = f["images"][entry_name]
        image = entry["image"][:]
        name = entry["image_name"][()].decode("utf-8")
        
        mirrors = []
        for mirror_name in entry["mirrors"]:
            mirrors.append(entry["mirrors"][mirror_name][:])

        print(f"{name}: image shape {image.shape}, mirrors: {[m.shape for m in mirrors]}")

image_2024-01-09_1500.jpg: image shape (480, 830), mirrors: [(10, 12), (13, 17), (21, 20), (19, 17), (20, 18), (19, 17), (19, 16), (19, 18), (20, 17), (20, 18), (20, 19), (20, 14), (21, 18), (18, 14), (21, 17), (21, 16), (21, 17), (22, 19), (21, 16), (21, 17), (21, 18), (20, 17), (21, 18), (18, 17), (18, 17), (18, 16), (18, 17), (17, 16), (17, 16), (17, 17), (16, 13), (16, 17), (16, 15), (17, 15), (16, 17), (19, 18), (17, 19), (17, 15), (18, 17), (18, 17), (18, 16), (19, 18), (17, 17), (17, 16), (18, 17), (18, 15), (17, 17), (18, 16), (19, 18), (18, 14), (19, 17), (18, 16), (19, 16), (19, 16), (20, 19), (19, 15), (20, 17), (16, 17), (19, 17), (20, 16), (19, 18), (17, 17), (17, 15), (17, 16), (18, 15), (18, 17), (19, 17), (19, 13), (14, 14), (20, 17), (20, 16), (20, 16), (20, 16), (22, 20), (21, 15), (22, 17), (22, 17), (23, 16), (24, 18), (16, 15), (17, 16), (18, 15), (17, 16), (18, 15), (18, 16), (17, 16), (17, 14), (18, 17), (18, 16), (18, 15), (16, 18), (17, 16), (18, 19), (17, 15),